### Amazon Sagemaker SDK 사용하기
필요한 파이선 라이브러리를 가져옵니다.

In [ ]:
bucket = 'sagemaker-handson-2-<userid>' # <userid> 대신 자신의 아이디를 넣어주세요
prefix = 'classification-colour-'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [21]:
import time
import json
import os

### 도커 이미지 빌드 및 배포
R에서 생성한 모델을 실행환경과 함께 도커 이미지로 만들고, ECR을 통해서 배포합니다.
도커 이미지를 처음 만드는 것이라면 5-10분정도 소요될 수 있습니다.

In [22]:
%%sh

# The name of our algorithm
algorithm_name=handson2

# set -e # stop if anything fails

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/18 : FROM ubuntu:16.04
 ---> b9e15a5d1e1a
Step 2/18 : MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>
 ---> Using cache
 ---> bb523a1d76c4
Step 3/18 : RUN apt-get -y update && apt-get install -y --no-install-recommends     wget     r-base     r-base-dev     ca-certificates     python2.7     python2.7-dev     python-pip     python-dev     python-virtualenv     git     vim     curl
 ---> Using cache
 ---> 7a2f8c051469
Step 4/18 : RUN apt-get install -y --no-install-recommends libssl-dev         libssh2-1-dev         libcurl4-openssl-dev         libcairo2-dev         libxt-dev         libfftw3-dev         libtiff5-dev         libxml2-dev
 ---> Using cache
 ---> bb86b1e5a30d
Step 5/18 : RUN pip install boto3
 ---> Using cache
 ---> 86660c2ef0ea
Step 6/18 : RUN pip install Pillow numpy scipy
 ---> Using cache
 ---> 7ed72ca8de16
Step 7/18 : RUN pip install --upgrade virtualenv
 ---> Using cache
 ---> 75b45e905eba
Step 8/18 : RUN pip install

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


---
## 호스팅
모델을 배포하기 위해서 생성한 도커이미지와 훈련한 모델파일의 경로를 지정해줍니다.

In [23]:
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
sm = boto3.client('sagemaker')

r_hosting_container = {
    'Image': '{}.dkr.ecr.{}.amazonaws.com/handson2:latest'.format(account, region),
    'ModelDataUrl': 's3://{}/model/model.tar.gz'.format(bucket)
}

r_job = prefix + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
create_model_response = sm.create_model(
    ModelName=r_job,
    ExecutionRoleArn=role,
    PrimaryContainer=r_hosting_container)

print(create_model_response['ModelArn'])

arn:aws:sagemaker:ap-northeast-2:156679781278:model/classification-colour-2018-09-07-14-09-34


엔드포인트 설정을 합니다.

In [24]:
r_endpoint_config = prefix + 'config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(r_endpoint_config)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=r_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'ModelName': r_job,
        'VariantName': 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

classification-colour-config-2018-09-07-14-09-54
Endpoint Config Arn: arn:aws:sagemaker:ap-northeast-2:156679781278:endpoint-config/classification-colour-config-2018-09-07-14-09-54


엔드포인트 설정대로 엔드포인트를 만듭니다. 해당 작업은 5-10분 정도 소요됩니다.

In [25]:
%%time

r_endpoint = prefix +'endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(r_endpoint)

create_endpoint_response = sm.create_endpoint(
    EndpointName=r_endpoint,
    EndpointConfigName=r_endpoint_config)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=r_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sm.get_waiter('endpoint_in_service').wait(EndpointName=r_endpoint)
finally:
    resp = sm.describe_endpoint(EndpointName=r_endpoint)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)

    if status != 'InService':
        raise Exception('Endpoint creation did not succeed')

classification-colour-endpoint-201809071409
arn:aws:sagemaker:ap-northeast-2:156679781278:endpoint/classification-colour-endpoint-201809071409
Status: Creating
Arn: arn:aws:sagemaker:ap-northeast-2:156679781278:endpoint/classification-colour-endpoint-201809071409
Status: InService
CPU times: user 144 ms, sys: 8 ms, total: 152 ms
Wall time: 5min 31s


---
## 예측하기
엔드포인트가 정상적으로 배포되고 동작하는지 확인합니다.

In [26]:
runtime = boto3.Session().client('runtime.sagemaker')

col = {"r": 73, "g": 84, "b": 116}
payload = json.dumps(col)
response = runtime.invoke_endpoint(EndpointName=r_endpoint,
                                   ContentType='application/json',
                                   Body=payload)

result = json.loads(response['Body'].read().decode())
result 

['blue']